In [ ]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define preprocessing pipeline
num_features = ["Age", "Weight", "Height", "Gestational Age", "systolic blood pressure",
                "diastolic blood pressure", "Heart Rate", "Glucose Level"]
cat_features = ["Pre-existing Conditions", "Diet Quality", "Exercise Level",
                "Smoking Status", "Mother’s Education"]

num_transformer = Pipeline(steps=[
    ("scaler", StandardScaler())
])
cat_transformer = Pipeline(steps=[
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(transformers=[
    ("num", num_transformer, num_features),
    ("cat", cat_transformer, cat_features)
])

# Save the preprocessing pipeline
with open("preprocessor.pkl", "wb") as f:
    pickle.dump(preprocessor, f)

# Define a sample model (replace with actual training process)
model = Sequential([
    Dense(64, activation="relu", input_shape=(100,)),  # Adjust input shape
    Dense(1, activation="sigmoid")
])

# Save the model
model.save("pregnancy_risk_model.h5")


In [ ]:
import pickle
import pandas as pd
from tensorflow.keras.models import load_model

# Load the preprocessing pipeline
with open("preprocessor.pkl", "rb") as f:
    loaded_preprocessor = pickle.load(f)

# Load the trained deep learning model
loaded_model = load_model("pregnancy_risk_model.h5")

# Prediction function
def predict_pregnancy_risk(
    age, weight, height, gestational_age, systolic_bp, diastolic_bp,
    heart_rate, glucose_level, pre_existing_conditions,
    diet_quality, exercise_level, smoking_status, mothers_education):

    # Create a DataFrame with input values
    input_data = pd.DataFrame({
        "Age": [age],
        "Weight": [weight],
        "Height": [height],
        "Gestational Age": [gestational_age],
        "systolic blood pressure": [systolic_bp],
        "diastolic blood pressure": [diastolic_bp],
        "Heart Rate": [heart_rate],
        "Glucose Level": [glucose_level],
        "Pre-existing Conditions": [pre_existing_conditions],
        "Diet Quality": [diet_quality],
        "Exercise Level": [exercise_level],
        "Smoking Status": [smoking_status],
        "Mother’s Education": [mothers_education]
    })

    # Preprocess input data
    input_transformed = loaded_preprocessor.transform(input_data)

    # Predict
    prediction = loaded_model.predict(input_transformed)
    risk = "High Risk" if prediction[0][0] > 0.5 else "Low Risk"

    return f"Prediction: {risk} ({prediction[0][0]:.2f})"

# Example usage
# print(predict_pregnancy_risk(28, 70, 165, 20, 120, 80, 72, 90, "None", "Good", "Moderate", "Non-smoker", "College Graduate"))
